In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [26]:
# Load the data
df = pd.read_csv('datasets/dataset.csv')

# Separate features and labels
X = df.iloc[:, :-7].values  # First 170 columns as features
y = df.iloc[:, -7:].values  # Last 7 columns as labels

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y_tensor = torch.tensor(y, dtype=torch.float32)


In [27]:
# Combine features and labels into a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# # 80% training and 20% testing split
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# 
# # DataLoader for training and testing
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [20]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 85, 128)  # 85 is derived from input size halved twice
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 7)  # 7 output labels

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = torch.relu(self.conv1(x))
        x = torch.max_pool1d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool1d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [28]:
import torch.nn.functional as F

class NetDropout(nn.Module):
    def __init__(self, n=32):
        super(NetDropout, self).__init__()
        self.n = n
        
        self.conv1 = nn.Conv1d(1, self.n, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout1d(p=0.3)
        self.conv2 = nn.Conv1d(self.n, self.n // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout1d(p=0.3)
        self.conv3 = nn.Conv1d(self.n // 2, self.n // 2, kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout1d(p=0.3)

        # Calculate the correct input size for the fully connected layer
        self.fc1 = nn.Linear((self.n // 2) * (170 // 8), 32)  # 170 // 8 due to three max_pool1d with kernel_size=2
        self.fc2 = nn.Linear(32, 7)
        
        self.lsftmx = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = F.max_pool1d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool1d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)

        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = torch.tanh(self.fc1(out))
        out = self.lsftmx(self.fc2(out))
        return out


In [17]:
# # Instantiate the model, define the loss function and the optimizer
# # model = SimpleCNN()
# model = NetDropout()
# criterion = nn.MSELoss()  # Assuming a regression task, otherwise use CrossEntropyLoss for classification
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# 
# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     for i, (inputs, labels) in enumerate(train_loader):
#         # Zero the parameter gradients
#         optimizer.zero_grad()
# 
#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
# 
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
# 
#         if (i + 1) % 10 == 0:
#             print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
# 
#     # Validation on the test set
#     model.eval()
#     with torch.no_grad():
#         total_loss = 0
#         for inputs, labels in test_loader:
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             total_loss += loss.item()
# 
#     print(f'Epoch [{epoch + 1}/{num_epochs}], Test Loss: {total_loss / len(test_loader):.4f}')


RuntimeError: shape '[-1, 256]' is invalid for input of size 336

In [43]:
# 80% training and 20% testing split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize the model, loss function, and optimizer
model = NetDropout().to(device)
criterion = nn.CrossEntropyLoss()  # Assuming multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(1000):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 1.5823680780229776
Epoch 2, Loss: 1.4320591618544865
Epoch 3, Loss: 1.3739072632615583
Epoch 4, Loss: 1.3425232931210176
Epoch 5, Loss: 1.3173266083654696
Epoch 6, Loss: 1.3005331078149978
Epoch 7, Loss: 1.2863831145920022
Epoch 8, Loss: 1.2759787367425697
Epoch 9, Loss: 1.268096109811407
Epoch 10, Loss: 1.2592021338913562
Epoch 11, Loss: 1.250825441449228
Epoch 12, Loss: 1.244237196815275
Epoch 13, Loss: 1.239513181856949
Epoch 14, Loss: 1.2356722292891384
Epoch 15, Loss: 1.2302210896554655
Epoch 16, Loss: 1.2249154154836697
Epoch 17, Loss: 1.2210352273970624
Epoch 18, Loss: 1.2197676194410254
Epoch 19, Loss: 1.217700432469375
Epoch 20, Loss: 1.211158982267345
Epoch 21, Loss: 1.2099617849083713
Epoch 22, Loss: 1.2080420916968018
Epoch 23, Loss: 1.2078871953226353
Epoch 24, Loss: 1.2011468010227175
Epoch 25, Loss: 1.2028066978184846
Epoch 26, Loss: 1.2027314125838942
Epoch 27, Loss: 1.1985429583022194
Epoch 28, Loss: 1.1972841438803359
Epoch 29, Loss: 1.1963543722229282


In [47]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
loader = test_loader
# loader = train_loader

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted class (highest log-probability)
        _, predicted = torch.max(outputs, 1)
        
        # print(outputs)
        # Calculate the number of correct predictions
        correct += (predicted == labels.argmax(dim=1)).sum().item()  # labels.argmax(dim=1) for one-hot encoded labels
        total += labels.size(0)

# Calculate average loss and accuracy
avg_test_loss = test_loss / len(loader)
accuracy = correct / total * 100

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')


Test Loss: 1.1119, Test Accuracy: 55.64%
